In [ ]:
import sys
sys.path.append('../build')

from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw
from libbem import *

In [ ]:
sp = Sphere( (0,0,0), 1)
mesh = Mesh( OCCGeometry(sp).GenerateMesh(maxh=0.2)).Curve(4)
# mesh = Mesh(unit_cube.GenerateMesh(maxh=0.3))

In [ ]:
u0 = 1/ sqrt( (x+1)**2 + (y+1)**2 + (z+1)**2 )

In [ ]:
fes = SurfaceL2(mesh, order=2, dual_mapping=False)
u,v = fes.TnT()
a = BilinearForm(fes)
SingleLayerPotential(a)
a.Assemble()
# print (a.mat)

In [ ]:
f = LinearForm (u0*v.Trace()*ds(bonus_intorder=3)).Assemble()

gfv = GridFunction(fes)
gfv.vec.data = a.mat.Inverse() * f.vec

In [ ]:
# gfv.Set (x, definedon=mesh.Boundaries(".*"))
Draw (gfv)

In [ ]:
gradu0 = CF( (u0.Diff(x), u0.Diff(y), u0.Diff(z)) )
n = specialcf.normal(3)
u1 = gradu0*n
Draw (u1, mesh, draw_vol=False)

In [ ]:
for t in Timers():
    if "SingleLayer" in t["name"]:
        print (t)

In [ ]:
print (Norm(gfv.vec))